In [0]:
# Gold from Silver (SQL-only, run in PySpark notebook)

# Silver tables
SILVER_CLIENT = "silver_client"
SILVER_PRODUCT = "silver_product"
SILVER_CHANNEL = "silver_channel"
SILVER_MKT_PREF = "silver_consent" #SCD2

# 4) C360_CLIENT_PRODUCT_SIGNALS (weekly)
# spec: ISO_WEEK_NUM + WEEK_END_DATE + 指标
# we use week_end_date = next_day(date, 'Sun'),  can be changed to Saturday
spark.sql(f"""
CREATE OR REPLACE TABLE c360_client_product_signals AS
WITH base AS (
SELECT
cast(client_id as bigint) AS client_id,
cast(product_code as string) AS product_code,
cast(product_event_type as string) AS product_event_type,
cast(product_event_date as date) AS d
FROM {SILVER_PRODUCT}
),
approved AS (
SELECT *
FROM base
WHERE product_event_type = 'Approved'
),
w AS (
SELECT
client_id,
weekofyear(d) AS iso_week_num,
next_day(d, 'Sun') AS week_end_date,
product_code
FROM approved
)
SELECT
client_id,
iso_week_num,
week_end_date,
max(CASE WHEN product_code LIKE 'CC%' THEN 1 ELSE 0 END) AS client_has_cc_ind,
max(CASE WHEN product_code LIKE 'CHQ%' THEN 1 ELSE 0 END) AS client_has_chq_ind,
max(CASE WHEN product_code LIKE 'SAV%' THEN 1 ELSE 0 END) AS client_has_sav_ind,
count(distinct product_code) AS client_num_products,
current_date() AS proc_dt
FROM w
GROUP BY client_id, iso_week_num, week_end_date
""")

# Quick checks
spark.sql("SELECT * FROM c360_client_product_signals LIMIT 10").display()